# 18 - Least Squares Optimization

## 18.1 - Problem Formulation

Given a matrix $A \in \mathbb{R}^{m \times n}$ (with $m \geq n$) and a vector $\mathbf{b} \in \mathbb{R}^m$, we are interested in finding a vector $x \in \mathbb{R}^n$ such that $A\mathbf{x} = \mathbf{b}$ if possible, or at least $A\mathbf{x} \approx \mathbf{b}$.

In the case where no such $\mathbf{x}$ satisfies the equation exactly, we define the residual as $r(\mathbf{x}) = \mathbf{b} - A\mathbf{x}$. The function $\phi(\mathbf{x})$ represents the square of the 2-norm of the residual

$$ \phi(\mathbf{x}) = ||\mathbf{b} - A\mathbf{x}||^2 = (\mathbf{b} - A\mathbf{x})^T (\mathbf{b} - A\mathbf{x}) $$

Our goal is to find a vector $\mathbf{x}$ which makes $A\mathbf{x}$ very close to $\mathbf{b}$, i.e., the $\mathbf{x}$ that minimizes the function $\phi(\mathbf{x})$.

The first necessary condition for minimization (setting the gradient to zero) leads to the normal equations

$$ \nabla \phi(\mathbf{x}) = 2A^T (\mathbf{b} - A\mathbf{x}) = 0 $$

which implies

$$ A^T A \mathbf{x} = A^T \mathbf{b} $$

This provides a closed-form solution $\mathbf{x}^* = (A^T A)^{-1} A^T \mathbf{b}$, assuming $A^T A$ is invertible. The Hessian matrix, given by $2A^T A$, is positive semi-definite, indicating that our solution is indeed a minimum.

## 18.2 - Singular Value Decomposition for Least Squares

Any matrix $A \in \mathbb{R}^{m \times n}$ can be decomposed using the singular value decomposition (SVD), expressed as $A = U \Sigma V^T$. Here, $U \in \mathbb{R}^{m \times m}$ and $V \in \mathbb{R}^{n \times n}$ are orthogonal matrices, and $\Sigma \in \mathbb{R}^{m \times n}$ is a diagonal matrix with non-negative real numbers on its diagonal.

For practical computations, particularly in the context of least squares problems, we often use the reduced form of SVD: $A = U_r \Sigma_r V^T_r$, where $U_r \in \mathbb{R}^{m \times r}$, $V_r \in \mathbb{R}^{n \times r}$, and $\Sigma_r \in \mathbb{R}^{r \times r}$ correspond to the first $r$ singular values, with $r$ being the rank of $A$. The least squares solution, then, is efficiently computed as

$$ \mathbf{x} \approx V_r \Sigma_r^{-1} U_r^T \mathbf{b} $$

where $\Sigma_r^{-1}$ is the diagonal matrix formed by taking the reciprocal of the non-zero elements in $\Sigma_r$.

Employing SVD to solve the least squares problem is particularly advantageous due to its numerical stability and its effectiveness in handling matrices that are rank-deficient or close to rank-deficient.

In [2]:
import numpy as np

# Define the matrix A and vector b
A = np.array([[2, 0], [-1, 1], [0, 2]])
b = np.array([1, 0, -1])

# Perform Singular Value Decomposition
# Use reduced SVD with option full_matrices=False
U, sigma, VT = np.linalg.svd(A, full_matrices=False)

# Create a diagonal matrix of the inverses of the singular values
Sigma_inv = np.diag(1 / sigma)

# Compute the pseudo-inverse of A
A_pinv = VT.T @ Sigma_inv @ U.T

# Compute the least squares solution using the pseudo-inverse
x = A_pinv @ b

print("Least squares solution x:", x)

Least squares solution x: [ 0.33333333 -0.33333333]


## 18.3 - Gradient Descent for Least Squares

Given a matrix $A \in \mathbb{R}^{m \times n}$ and a vector $\mathbf{b} \in \mathbb{R}^m$, we aim to find a vector $\mathbf{x} \in \mathbb{R}^n$ that minimizes the objective function:

$$ \phi(\mathbf{x}) = ||A\mathbf{x} - \mathbf{b}||^2 $$

The gradient of the objective function $\phi$ with respect to $\mathbf{x}$ is given by

$$ \nabla \phi(\mathbf{x}) = 2A^T (A\mathbf{x} - \mathbf{b}) $$

**Gradient Descent Algorithm**
- **Initialization**: Start with an initial guess for $\mathbf{x}$, typically $\mathbf{x}_0 = 0$ or a random vector.
- **Iteration**: Update $\mathbf{x}$ iteratively using the formula:

    $$ \mathbf{x}_{\text{new}} = \mathbf{x}_{\text{old}} - \alpha \nabla \phi(\mathbf{x}_{\text{old}}) $$

    where $\alpha$ is the learning rate, a small positive scalar that controls the step size.
- **Convergence Check**: Repeat the iteration until $\nabla \phi(\mathbf{x})$ is sufficiently small or a maximum number of iterations is reached.

In [3]:
# Parameters for gradient descent
alpha = 0.01        # Learning rate
max_iter = 1000     # Max number of iterations
tolerance = 1e-6    # Convergence tolerance

# Initial guess for x
x = np.zeros(A.shape[1])

# Gradient descent iteration
for i in range(max_iter):
    gradient = 2 * A.T @ (A @ x - b)
    x -= alpha * gradient
    if np.linalg.norm(gradient) < tolerance:
        break

print("Solution x:", x)

Solution x: [ 0.33333328 -0.33333328]
